# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nhl_df=pd.read_csv("assets/nhl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]
cities.columns = ['Metropolitan area', 'Population', 'NFL', 'MLB', 'NBA','NHL']

# Take out the [note x] part
cities['NHL'] = cities['NHL'].str.replace('\[.*\]','', regex=True)

def separateOnCase (string):
    if re.search('[a-z][A-Z]', string):
        subfinal = []
        fstring = ''
        mylist = re.split(r'([A-Z])', string)
        # Remove the 0th element
        mylist.pop(0)
        for i in range(0,len(mylist),2):
            subfinal.append(mylist[i]+mylist[i+1])
        for word in subfinal:
            fstring = fstring + '_' + word
        return fstring.lstrip('_')
    elif not re.search('[A-Z]',string):
        return np.nan
    else:
        return string

def countTeams (df, col):
    for index,team in enumerate(col):
        x = team.count('_')
        for i in range(x):
            df = df.append(df.iloc[index])
    return df.reset_index().drop('index',1)

def deDuplicateTeams(col):
    count_list = []
    final = []
    for teams in col:
        if teams.find('_') != -1:
            count_list.append(teams)
            final.append(teams.split('_')[count_list.count(teams)-1])
        else:
            final.append(teams)
    return final
            
cities['NHL'] = cities['NHL'].apply(separateOnCase)
df1 = cities[['Population','Metropolitan area','NHL']].dropna()
df1 = countTeams(df1,df1['NHL'])
df1['NHL'] = deDuplicateTeams(df1['NHL'])
df1

,Population,Metropolitan area,NHL
0,20153634,New York City,Rangers
1,13310447,Los Angeles,Kings
2,6657982,San Francisco Bay Area,Sharks
3,9512999,Chicago,Blackhawks
4,7233323,Dallas–Fort Worth,Stars
5,6131977,"Washington, D.C.",Capitals
6,6070500,Philadelphia,Flyers
7,4794447,Boston,Bruins
8,3551036,Minneapolis–Saint Paul,Wild
9,2853077,Denver,Avalanche


In [2]:
nhl_df=pd.read_csv("assets/nhl.csv")
nhl_df = nhl_df[nhl_df['year'] == 2018]
nhl_df['team'] = nhl_df['team'].apply(lambda string: string.replace('*',''))
nhl_df = nhl_df.drop(index = [0,9,18,26]).reset_index().drop('index',1)

def separateCity(col, Allteams):
    allList = []
    final = []
    for team in Allteams:
        allList.append(team.split(' '))
    allList = list(np.concatenate(allList).flat)
    for teams in col:
        l = teams.split(' ')
        temp = ''
        for word in l:
            if word in allList:
                temp = temp + ' ' + word
        final.append(temp.lstrip(' '))
    return final

nhl_df['team'] = separateCity(nhl_df['team'],df1['NHL'])
newdf = df1.merge(nhl_df, right_on='team', left_on = 'NHL')
newdf = newdf[['NHL','Metropolitan area','Population','W','L']]
cols = ['Population','W','L']
for col in cols:
    newdf[col] = pd.to_numeric(newdf[col])
    
finaldf = newdf.groupby('Metropolitan area', sort= False).agg({'Population': np.mean , 'W': np.sum, 'L': np.sum})
finaldf['W/L'] = finaldf['W']/(finaldf['W']+finaldf['L'])
finaldf.reset_index(inplace = True)
finaldf

,Metropolitan area,Population,W,L,W/L
0,New York City,20153634,113,105,0.518349
1,Los Angeles,13310447,89,54,0.622378
2,San Francisco Bay Area,6657982,45,27,0.625000
3,Chicago,9512999,33,39,0.458333
4,Dallas–Fort Worth,7233323,42,32,0.567568
5,"Washington, D.C.",6131977,49,26,0.653333
6,Philadelphia,6070500,42,26,0.617647
7,Boston,4794447,50,20,0.714286
8,Minneapolis–Saint Paul,3551036,45,26,0.633803
9,Denver,2853077,43,30,0.589041


In [3]:
population_by_region = list(finaldf['Population'])
win_loss_by_region = list(finaldf['W/L'])
stats.pearsonr(population_by_region, win_loss_by_region)[0]

0.012308996455744285

In [4]:
def nhl_correlation():
    import pandas as pd
    import numpy as np
    import scipy.stats as stats
    import re

    nhl_df=pd.read_csv("assets/nhl.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    cities.columns = ['Metropolitan area', 'Population', 'NFL', 'MLB', 'NBA','NHL']
    
    # Take out the [note x] part
    
    cities['NHL'] = cities['NHL'].str.replace('\[.*\]','', regex=True)

    def separateOnCase (string):
        if re.search('[a-z][A-Z]', string):
            subfinal = []
            fstring = ''
            mylist = re.split(r'([A-Z])', string)
            # Remove the 0th element
            mylist.pop(0)
            for i in range(0,len(mylist),2):
                subfinal.append(mylist[i]+mylist[i+1])
            for word in subfinal:
                fstring = fstring + '_' + word
            return fstring.lstrip('_')
        elif not re.search('[A-Z]',string):
            return np.nan
        else:
            return string

    def countTeams (df, col):
        for index,team in enumerate(col):
            x = team.count('_')
            for i in range(x):
                df = df.append(df.iloc[index])
        return df.reset_index().drop('index',1)

    def deDuplicateTeams(col):
        count_list = []
        final = []
        for teams in col:
            if teams.find('_') != -1:
                count_list.append(teams)
                final.append(teams.split('_')[count_list.count(teams)-1])
            else:
                final.append(teams)
        return final

    cities['NHL'] = cities['NHL'].apply(separateOnCase)
    df1 = cities[['Population','Metropolitan area','NHL']].dropna()
    df1 = countTeams(df1,df1['NHL'])
    df1['NHL'] = deDuplicateTeams(df1['NHL'])
    
    ###########################################################################################
    
    nhl_df = nhl_df[nhl_df['year'] == 2018]
    nhl_df['team'] = nhl_df['team'].apply(lambda string: string.replace('*',''))
    nhl_df = nhl_df.drop(index = [0,9,18,26]).reset_index().drop('index',1)

    def separateCity(col, Allteams):
        allList = []
        final = []
        for team in Allteams:
            allList.append(team.split(' '))
        allList = list(np.concatenate(allList).flat)
        for teams in col:
            l = teams.split(' ')
            temp = ''
            for word in l:
                if word in allList:
                    temp = temp + ' ' + word
            final.append(temp.lstrip(' '))
        return final

    nhl_df['team'] = separateCity(nhl_df['team'],df1['NHL'])
    newdf = df1.merge(nhl_df, right_on='team', left_on = 'NHL')
    newdf = newdf[['NHL','Metropolitan area','Population','W','L']]
    cols = ['Population','W','L']
    for col in cols:
        newdf[col] = pd.to_numeric(newdf[col])

    finaldf = newdf.groupby('Metropolitan area', sort= False).agg({'Population': np.mean , 'W': np.sum, 'L': np.sum})
    finaldf['W/L'] = finaldf['W']/(finaldf['W']+finaldf['L'])
    finaldf = finaldf.reset_index()
    
    ################################################################################################
    
    population_by_region = list(finaldf['Population']) # pass in metropolitan area population from cities
    win_loss_by_region = list(finaldf['W/L']) # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    
    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [5]:
def separateOnCase (string):
    if re.search('[a-z][A-Z]', string):
        subfinal = []
        fstring = ''
        mylist = re.split(r'([A-Z])', string)
        # Remove the 0th element
        mylist.pop(0)
        for i in range(0,len(mylist),2):
            subfinal.append(mylist[i]+mylist[i+1])
        for word in subfinal:
            fstring = fstring + '_' + word
        return fstring.lstrip('_')
    elif not re.search('[A-Z]|[0-9]',string):
        return np.nan
    else:
        return string

def countTeams (df, col):
    for index,team in enumerate(col):
        x = team.count('_')
        for i in range(x):
            df = df.append(df.iloc[index])
    return df.reset_index().drop('index',1)

def deDuplicateTeams(col):
    count_list = []
    final = []
    for teams in col:
        if teams.find('_') != -1:
            count_list.append(teams)
            final.append(teams.split('_')[count_list.count(teams)-1])
        else:
            final.append(teams)
    return final

In [6]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nba_df=pd.read_csv("assets/nba.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]
cities.columns = ['Metropolitan area', 'Population', 'NFL', 'MLB', 'NBA','NHL']

# Take out the [note x] part
cities['NBA'] = cities['NBA'].str.replace('\[.*\]','', regex=True)
cities['NBA'] = cities['NBA'].apply(separateOnCase)
df2 = cities[['Population','Metropolitan area','NBA']].dropna()
df2 = countTeams(df2,df2['NBA'])
df2 = df2.reset_index().drop('index',1)
df2['NBA'] = deDuplicateTeams(df2['NBA'])
df2

,Population,Metropolitan area,NBA
0,20153634,New York City,Knicks
1,13310447,Los Angeles,Lakers
2,6657982,San Francisco Bay Area,Warriors
3,9512999,Chicago,Bulls
4,7233323,Dallas–Fort Worth,Mavericks
5,6131977,"Washington, D.C.",Wizards
6,6070500,Philadelphia,76ers
7,4794447,Boston,Celtics
8,3551036,Minneapolis–Saint Paul,Timberwolves
9,2853077,Denver,Nuggets


In [7]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re
nba_df=pd.read_csv("assets/nba.csv")

def separateCity(col, Allteams):
    allList = []
    final = []
    for team in Allteams:
        allList.append(team.split(' '))
    allList = list(np.concatenate(allList).flat)
    for teams in col:
        l = teams.split(' ')
        temp = ''
        for word in l:
            if word in allList:
                temp = temp + ' ' + word
        final.append(temp.lstrip(' '))
    return final

nba_df = nba_df[nba_df['year'] == 2018]
nba_df['team'] = nba_df['team'].apply(lambda string: string.replace('*',''))
nba_df['team'] = nba_df['team'].apply(lambda string: re.sub('\(\d*\)','',string))
nba_df['team'] = nba_df['team'].apply(lambda string: string.strip())
nba_df = nba_df[['team','W','L']]

nba_df['team'] = separateCity(nba_df['team'],df2['NBA'])
newdf2 = df2.merge(nba_df, right_on='team', left_on = 'NBA')
newdf2 = newdf2[['NBA','Metropolitan area','Population','W','L']]
cols = ['Population','W','L']
for col in cols:
    newdf2[col] = pd.to_numeric(newdf2[col])

finaldf = newdf2.groupby('Metropolitan area', sort= False).agg({'Population': np.mean , 'W': np.sum, 'L': np.sum})
finaldf['W/L'] = finaldf['W']/(finaldf['W']+finaldf['L'])
finaldf = finaldf.reset_index()
finaldf

,Metropolitan area,Population,W,L,W/L
0,New York City,20153634,57,107,0.347561
1,Los Angeles,13310447,77,87,0.469512
2,San Francisco Bay Area,6657982,58,24,0.707317
3,Chicago,9512999,27,55,0.329268
4,Dallas–Fort Worth,7233323,24,58,0.292683
5,"Washington, D.C.",6131977,43,39,0.524390
6,Philadelphia,6070500,52,30,0.634146
7,Boston,4794447,55,27,0.670732
8,Minneapolis–Saint Paul,3551036,47,35,0.573171
9,Denver,2853077,46,36,0.560976


In [8]:
population_by_region = list(finaldf['Population'])
win_loss_by_region = list(finaldf['W/L'])
stats.pearsonr(population_by_region, win_loss_by_region)[0]

-0.1765716025284462

In [22]:
def nba_correlation():
    import pandas as pd
    import numpy as np
    import scipy.stats as stats
    import re

    nba_df=pd.read_csv("assets/nba.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    cities.columns = ['Metropolitan area', 'Population', 'NFL', 'MLB', 'NBA','NHL']
    
    def separateOnCase (string):
        if re.search('[a-z][A-Z]', string):
            subfinal = []
            fstring = ''
            mylist = re.split(r'([A-Z])', string)
            # Remove the 0th element
            mylist.pop(0)
            for i in range(0,len(mylist),2):
                subfinal.append(mylist[i]+mylist[i+1])
            for word in subfinal:
                fstring = fstring + '_' + word
            return fstring.lstrip('_')
        elif not re.search('[A-Z]|[0-9]',string):
            return np.nan
        else:
            return string

    def countTeams (df, col):
        for index,team in enumerate(col):
            x = team.count('_')
            for i in range(x):
                df = df.append(df.iloc[index])
        return df.reset_index().drop('index',1)

    def deDuplicateTeams(col):
        count_list = []
        final = []
        for teams in col:
            if teams.find('_') != -1:
                count_list.append(teams)
                final.append(teams.split('_')[count_list.count(teams)-1])
            else:
                final.append(teams)
        return final
    
    def separateCity(col, Allteams):
        allList = []
        final = []
        for team in Allteams:
            allList.append(team.split(' '))
        allList = list(np.concatenate(allList).flat)
        for teams in col:
            l = teams.split(' ')
            temp = ''
            for word in l:
                if word in allList:
                    temp = temp + ' ' + word
            final.append(temp.lstrip(' '))
        return final
    

    # Take out the [note x] part
    cities['NBA'] = cities['NBA'].str.replace('\[.*\]','', regex=True)
    cities['NBA'] = cities['NBA'].apply(separateOnCase)
    df2 = cities[['Population','Metropolitan area','NBA']].dropna()
    df2 = countTeams(df2,df2['NBA'])
    df2 = df2.reset_index().drop('index',1)
    df2['NBA'] = deDuplicateTeams(df2['NBA'])
    
    ###################################################################################################################
    
    nba_df = nba_df[nba_df['year'] == 2018]
    nba_df['team'] = nba_df['team'].apply(lambda string: string.replace('*',''))
    nba_df['team'] = nba_df['team'].apply(lambda string: re.sub('\(\d*\)','',string))
    nba_df['team'] = nba_df['team'].apply(lambda string: string.strip())
    nba_df = nba_df[['team','W','L']]

    nba_df['team'] = separateCity(nba_df['team'],df2['NBA'])
    newdf2 = df2.merge(nba_df, right_on='team', left_on = 'NBA')
    newdf2 = newdf2[['NBA','Metropolitan area','Population','W','L']]
    cols = ['Population','W','L']
    for col in cols:
        newdf2[col] = pd.to_numeric(newdf2[col])

    finaldf = newdf2.groupby('Metropolitan area', sort= False).agg({'Population': np.mean , 'W': np.sum, 'L': np.sum})
    finaldf['W/L'] = finaldf['W']/(finaldf['W']+finaldf['L'])
    finaldf = finaldf.reset_index()
    
    ###############################################################################################################
    
    population_by_region = list(finaldf['Population']) # pass in metropolitan area population from cities
    win_loss_by_region = list(finaldf['W/L']) # pass in win/loss ratio from nba_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [49]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

def separateOnCase (string):
    if re.search('[a-z][A-Z]', string):
        subfinal = []
        fstring = ''
        mylist = re.split(r'([A-Z])', string)
        # Remove the 0th element
        mylist.pop(0)
        for i in range(0,len(mylist),2):
            if ' ' in mylist[i+1]:
                subfinal.append(mylist[i]+mylist[i+1]+mylist[i+2]+mylist[i+3])
            elif ' ' in mylist[i-1]:
                pass
            else:
                subfinal.append(mylist[i]+mylist[i+1])
        for word in subfinal:
            fstring = fstring + '_' + word
        return fstring.lstrip('_')
    elif not re.search('[A-Z]|[0-9]',string):
        return np.nan
    else:
        return string
    
mlb_df=pd.read_csv("assets/mlb.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]
cities.columns = ['Metropolitan area', 'Population', 'NFL', 'MLB', 'NBA','NHL']

# Take out the [note x] part
cities['MLB'] = cities['MLB'].str.replace('\[.*\]','', regex=True)
cities['MLB'] = cities['MLB'].apply(separateOnCase)
df3 = cities[['Population','Metropolitan area','MLB']].dropna()
df3 = countTeams(df3,df3['MLB'])
df3 = df3.reset_index().drop('index',1)
df3['MLB'] = deDuplicateTeams(df3['MLB'])
df3

,Population,Metropolitan area,MLB
0,20153634,New York City,Yankees
1,13310447,Los Angeles,Dodgers
2,6657982,San Francisco Bay Area,Giants
3,9512999,Chicago,Cubs
4,7233323,Dallas–Fort Worth,Rangers
5,6131977,"Washington, D.C.",Nationals
6,6070500,Philadelphia,Phillies
7,4794447,Boston,Red Sox
8,3551036,Minneapolis–Saint Paul,Twins
9,2853077,Denver,Rockies


In [50]:
mlb_df = mlb_df[mlb_df['year'] == 2018]
mlb_df['team'] = mlb_df['team'].apply(lambda string: string.replace('*',''))
mlb_df['team'] = mlb_df['team'].apply(lambda string: re.sub('\(\d*\)','',string))
mlb_df['team'] = mlb_df['team'].apply(lambda string: string.strip())
mlb_df = mlb_df[['team','W','L']]

mlb_df['team'] = separateCity(mlb_df['team'],df3['MLB'])
newdf3 = df3.merge(mlb_df, right_on='team', left_on = 'MLB')
newdf3 = newdf3[['MLB','Metropolitan area','Population','W','L']]
cols = ['Population','W','L']
for col in cols:
    newdf3[col] = pd.to_numeric(newdf3[col])
newdf3
finaldf = newdf3.groupby('Metropolitan area', sort= False).agg({'Population': np.mean , 'W': np.sum, 'L': np.sum})
finaldf['W/L'] = finaldf['W']/(finaldf['W']+finaldf['L'])
finaldf = finaldf.reset_index()
finaldf

,Metropolitan area,Population,W,L,W/L
0,New York City,20153634,177,147,0.546296
1,Los Angeles,13310447,172,153,0.529231
2,San Francisco Bay Area,6657982,170,154,0.524691
3,Chicago,9512999,157,168,0.483077
4,Dallas–Fort Worth,7233323,67,95,0.413580
5,"Washington, D.C.",6131977,82,80,0.506173
6,Philadelphia,6070500,80,82,0.493827
7,Boston,4794447,108,54,0.666667
8,Minneapolis–Saint Paul,3551036,78,84,0.481481
9,Denver,2853077,91,72,0.558282


In [51]:
population_by_region = list(finaldf['Population'])
win_loss_by_region = list(finaldf['W/L'])
stats.pearsonr(population_by_region, win_loss_by_region)[0]

0.1505230448710485

In [54]:
def mlb_correlation(): 
    import pandas as pd
    import numpy as np
    import scipy.stats as stats
    import re

    mlb_df=pd.read_csv("assets/mlb.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    cities.columns = ['Metropolitan area', 'Population', 'NFL', 'MLB', 'NBA','NHL']

    def separateOnCase (string):
        if re.search('[a-z][A-Z]', string):
            subfinal = []
            fstring = ''
            mylist = re.split(r'([A-Z])', string)
            # Remove the 0th element
            mylist.pop(0)
            for i in range(0,len(mylist),2):
                if ' ' in mylist[i+1]:
                    subfinal.append(mylist[i]+mylist[i+1]+mylist[i+2]+mylist[i+3])
                elif ' ' in mylist[i-1]:
                    pass
                else:
                    subfinal.append(mylist[i]+mylist[i+1])
            for word in subfinal:
                fstring = fstring + '_' + word
            return fstring.lstrip('_')
        elif not re.search('[A-Z]|[0-9]',string):
            return np.nan
        else:
            return string

    def countTeams (df, col):
        for index,team in enumerate(col):
            x = team.count('_')
            for i in range(x):
                df = df.append(df.iloc[index])
        return df.reset_index().drop('index',1)

    def deDuplicateTeams(col):
        count_list = []
        final = []
        for teams in col:
            if teams.find('_') != -1:
                count_list.append(teams)
                final.append(teams.split('_')[count_list.count(teams)-1])
            else:
                final.append(teams)
        return final
    
    def separateCity(col, Allteams):
        allList = []
        final = []
        for team in Allteams:
            allList.append(team.split(' '))
        allList = list(np.concatenate(allList).flat)
        for teams in col:
            l = teams.split(' ')
            temp = ''
            for word in l:
                if word in allList:
                    temp = temp + ' ' + word
            final.append(temp.lstrip(' '))
        return final
    
    # Take out the [note x] part
    cities['MLB'] = cities['MLB'].str.replace('\[.*\]','', regex=True)
    cities['MLB'] = cities['MLB'].apply(separateOnCase)
    df3 = cities[['Population','Metropolitan area','MLB']].dropna()
    df3 = countTeams(df3,df3['MLB'])
    df3 = df3.reset_index().drop('index',1)
    df3['MLB'] = deDuplicateTeams(df3['MLB'])
    
    ##################################################################################################################
    
    mlb_df = mlb_df[mlb_df['year'] == 2018]
    mlb_df['team'] = mlb_df['team'].apply(lambda string: string.replace('*',''))
    mlb_df['team'] = mlb_df['team'].apply(lambda string: re.sub('\(\d*\)','',string))
    mlb_df['team'] = mlb_df['team'].apply(lambda string: string.strip())
    mlb_df = mlb_df[['team','W','L']]

    mlb_df['team'] = separateCity(mlb_df['team'],df3['MLB'])
    newdf3 = df3.merge(mlb_df, right_on='team', left_on = 'MLB')
    newdf3 = newdf3[['MLB','Metropolitan area','Population','W','L']]
    cols = ['Population','W','L']
    for col in cols:
        newdf3[col] = pd.to_numeric(newdf3[col])

    finaldf = newdf3.groupby('Metropolitan area', sort= False).agg({'Population': np.mean , 'W': np.sum, 'L': np.sum})
    finaldf['W/L'] = finaldf['W']/(finaldf['W']+finaldf['L'])
    finaldf = finaldf.reset_index()
    
    ##############################################################################################################
    
    population_by_region = list(finaldf['Population']) # pass in metropolitan area population from cities
    win_loss_by_region = list(finaldf['W/L']) # pass in win/loss ratio from mlb_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

In [55]:
mlb_correlation()

0.1505230448710485

## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [90]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]
cities.columns = ['Metropolitan area', 'Population', 'NFL', 'MLB', 'NBA','NHL']


def separateOnCase (string):
    if re.search('[a-z][A-Z]', string):
        subfinal = []
        fstring = ''
        mylist = re.split(r'([A-Z]|4)', string)
        # Remove the 0th element
        mylist.pop(0)
        for i in range(0,len(mylist),2):
            if ' ' in mylist[i+1]:
                subfinal.append(mylist[i]+mylist[i+1]+mylist[i+2]+mylist[i+3])
            elif ' ' in mylist[i-1]:
                pass
            else:
                subfinal.append(mylist[i]+mylist[i+1])
        for word in subfinal:
            fstring = fstring + '_' + word
        return fstring.lstrip('_')
    elif not re.search('[A-Z]|[0-9]',string):
        return np.nan
    else:
        return string

# Take out the [note x] part
cities['NFL'] = cities['NFL'].str.replace('\[.*\]','', regex=True)
cities['NFL'] = cities['NFL'].apply(separateOnCase)
df4 = cities[['Population','Metropolitan area','NFL']].dropna()
df4 = countTeams(df4,df4['NFL'])
df4 = df4.reset_index().drop('index',1)
df4['NFL'] = deDuplicateTeams(df4['NFL'])
df4

,Population,Metropolitan area,NFL
0,20153634,New York City,Giants
1,13310447,Los Angeles,Rams
2,6657982,San Francisco Bay Area,49ers
3,9512999,Chicago,Bears
4,7233323,Dallas–Fort Worth,Cowboys
5,6131977,"Washington, D.C.",Redskins
6,6070500,Philadelphia,Eagles
7,4794447,Boston,Patriots
8,3551036,Minneapolis–Saint Paul,Vikings
9,2853077,Denver,Broncos


In [91]:
nfl_df = nfl_df[nfl_df['year'] == 2018]
nfl_df['team'] = nfl_df['team'].apply(lambda string: string.replace('*',''))
nfl_df['team'] = nfl_df['team'].apply(lambda string: string.replace('+',''))
nfl_df['team'] = nfl_df['team'].apply(lambda string: re.sub('\(\d*\)','',string))
nfl_df['team'] = nfl_df['team'].apply(lambda string: string.strip())
nfl_df = nfl_df[['team','W','L']]

nfl_df['team'] = separateCity(nfl_df['team'],df4['NFL'])
newdf4 = df4.merge(nfl_df, right_on='team', left_on = 'NFL')
newdf4 = newdf4[['NFL','Metropolitan area','Population','W','L']]
cols = ['Population','W','L']
for col in cols:
    newdf4[col] = pd.to_numeric(newdf4[col])

finaldf = newdf4.groupby('Metropolitan area', sort= False).agg({'Population': np.mean , 'W': np.sum, 'L': np.sum})
finaldf['W/L'] = finaldf['W']/(finaldf['W']+finaldf['L'])
finaldf = finaldf.reset_index()
finaldf

,Metropolitan area,Population,W,L,W/L
0,New York City,20153634,9,23,0.281250
1,Los Angeles,13310447,25,7,0.781250
2,San Francisco Bay Area,6657982,8,24,0.250000
3,Chicago,9512999,12,4,0.750000
4,Dallas–Fort Worth,7233323,10,6,0.625000
5,"Washington, D.C.",6131977,7,9,0.437500
6,Philadelphia,6070500,9,7,0.562500
7,Boston,4794447,11,5,0.687500
8,Minneapolis–Saint Paul,3551036,8,7,0.533333
9,Denver,2853077,6,10,0.375000


In [92]:
population_by_region = list(finaldf['Population'])
win_loss_by_region = list(finaldf['W/L'])
stats.pearsonr(population_by_region, win_loss_by_region)[0]

0.004922112149349386

In [93]:
def nfl_correlation(): 
    import pandas as pd
    import numpy as np
    import scipy.stats as stats
    import re

    def separateOnCase (string):
        if re.search('[a-z][A-Z]', string):
            subfinal = []
            fstring = ''
            mylist = re.split(r'([A-Z]|4)', string)
            # Remove the 0th element
            mylist.pop(0)
            for i in range(0,len(mylist),2):
                if ' ' in mylist[i+1]:
                    subfinal.append(mylist[i]+mylist[i+1]+mylist[i+2]+mylist[i+3])
                elif ' ' in mylist[i-1]:
                    pass
                else:
                    subfinal.append(mylist[i]+mylist[i+1])
            for word in subfinal:
                fstring = fstring + '_' + word
            return fstring.lstrip('_')
        elif not re.search('[A-Z]|[0-9]',string):
            return np.nan
        else:
            return string

    def countTeams (df, col):
        for index,team in enumerate(col):
            x = team.count('_')
            for i in range(x):
                df = df.append(df.iloc[index])
        return df.reset_index().drop('index',1)

    def deDuplicateTeams(col):
        count_list = []
        final = []
        for teams in col:
            if teams.find('_') != -1:
                count_list.append(teams)
                final.append(teams.split('_')[count_list.count(teams)-1])
            else:
                final.append(teams)
        return final
    
    def separateCity(col, Allteams):
        allList = []
        final = []
        for team in Allteams:
            allList.append(team.split(' '))
        allList = list(np.concatenate(allList).flat)
        for teams in col:
            l = teams.split(' ')
            temp = ''
            for word in l:
                if word in allList:
                    temp = temp + ' ' + word
            final.append(temp.lstrip(' '))
        return final
    
    nfl_df=pd.read_csv("assets/nfl.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    cities.columns = ['Metropolitan area', 'Population', 'NFL', 'MLB', 'NBA','NHL']

    # Take out the [note x] part
    cities['NFL'] = cities['NFL'].str.replace('\[.*\]','', regex=True)
    cities['NFL'] = cities['NFL'].apply(separateOnCase)
    df4 = cities[['Population','Metropolitan area','NFL']].dropna()
    df4 = countTeams(df4,df4['NFL'])
    df4 = df4.reset_index().drop('index',1)
    df4['NFL'] = deDuplicateTeams(df4['NFL'])

    ############################################################################################################
    
    nfl_df = nfl_df[nfl_df['year'] == 2018]
    nfl_df['team'] = nfl_df['team'].apply(lambda string: string.replace('*',''))
    nfl_df['team'] = nfl_df['team'].apply(lambda string: string.replace('+',''))
    nfl_df['team'] = nfl_df['team'].apply(lambda string: re.sub('\(\d*\)','',string))
    nfl_df['team'] = nfl_df['team'].apply(lambda string: string.strip())
    nfl_df = nfl_df[['team','W','L']]

    nfl_df['team'] = separateCity(nfl_df['team'],df4['NFL'])
    newdf4 = df4.merge(nfl_df, right_on='team', left_on = 'NFL')
    newdf4 = newdf4[['NFL','Metropolitan area','Population','W','L']]
    cols = ['Population','W','L']
    for col in cols:
        newdf4[col] = pd.to_numeric(newdf4[col])

    finaldf = newdf4.groupby('Metropolitan area', sort= False).agg({'Population': np.mean , 'W': np.sum, 'L': np.sum})
    finaldf['W/L'] = finaldf['W']/(finaldf['W']+finaldf['L'])
    finaldf = finaldf.reset_index()
    
    #############################################################################################################
    
    population_by_region = list(finaldf['Population'])
    win_loss_by_region = list(finaldf['W/L'])

    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

In [94]:
nfl_correlation()

0.004922112149349386

## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [15]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
nhl_df=pd.read_csv("assets/nhl.csv")
nba_df=pd.read_csv("assets/nba.csv")
nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def sports_team_performance():
    # YOUR CODE HERE
    raise NotImplementedError()
    
    # Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and
    # df.loc["NFL","NFL"] should return np.nan
    sports = ['NFL', 'NBA', 'NHL', 'MLB']
    p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)
    
    assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NHL p-value should be around 0.02"
    assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
    return p_values